# Feature Engineering

In [1]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

from tqdm import tqdm

import time
import torch
import gc

pd.set_option('display.max_columns', None)

## Daten laden

In [47]:
IS_TRAINING = False
if IS_TRAINING:
    FILE = '../../data/processed/train_series_split_normalized.parquet'
else:
    FILE = '../../data/processed/validation_series_split_normalized.parquet'
series = pd.read_parquet(FILE, columns=['num_series_id'])

## Features

In [48]:
LAGS_FUTURE = [f"t_lag_{i}" for i in range(-1, -25, -1)]
LAGS_PAST = [f"t_lag_{i}" for i in range(1, 25)]
FEATURES = ['t_0', *LAGS_PAST, *LAGS_FUTURE]

In [49]:
len(FEATURES)

49

## Neue Features erstellen

In [50]:
folder = "train/" if IS_TRAINING else "validation/"
file_path_prefix = "../../data/processed/transformer/" + folder

def save_chunk(num_series_id, chunk):
    series_length, series_columns = chunk[FEATURES].values.shape
    X = torch.from_numpy(np.vstack(np.ravel(chunk[FEATURES].values))
                           .reshape(series_length, series_columns, 2)).to(torch.float32)
    torch.save(X, file_path_prefix + str(num_series_id) + ".pt")

In [51]:
def make_features_chunk(num_series_id):
    df = pd.read_parquet(FILE, filters=[('num_series_id','=',num_series_id)])
    df['t_0'] = df[['anglez', 'enmo']].values.tolist()

    for i in range(1, 25):
        df[f'anglez_lag_{i}'] = df["anglez"].shift(i).bfill()
        df[f'enmo_lag_{i}'] = df["enmo"].shift(i).bfill()
        df[f't_lag_{i}'] = df[[f'anglez_lag_{i}', f'enmo_lag_{i}']].values.tolist()
        df = df.drop(columns=[f'anglez_lag_{i}', f'enmo_lag_{i}'])

    for i in range(-1, -25, -1):
        df[f'anglez_lag_{i}'] = df["anglez"].shift(i).ffill()
        df[f'enmo_lag_{i}'] = df["enmo"].shift(i).ffill()
        df[f't_lag_{i}'] = df[[f'anglez_lag_{i}', f'enmo_lag_{i}']].values.tolist()
        df = df.drop(columns=[f'anglez_lag_{i}', f'enmo_lag_{i}'])
    
    return df.reset_index(drop=True)

In [52]:
def make_features(series):
    overview_data = []
    abort = False

    for num_series_id in tqdm(series.num_series_id.unique()):
        chunk = make_features_chunk(num_series_id)
        save_chunk(num_series_id, chunk)

        overview_data.append(
            chunk[['num_series_id', 'awake']].reset_index().rename(columns={'index':'series_index'}).copy()[['num_series_id', 'awake', 'series_index']]
        )

        del chunk
        gc.collect()
    
    return pd.concat(overview_data).reset_index(drop=True)

In [53]:
start_time = time.time()
overview = make_features(series)
print(f'Feature Engineering took {time.time() - start_time} seconds')

100%|██████████████████████████████████████████████████████████████████████████████████| 54/54 [43:01<00:00, 47.81s/it]


Feature Engineering took 2582.5897183418274 seconds


## Overview speichern

In [54]:
overview

,num_series_id,awake,series_index
0,7,1,0
1,7,1,1
2,7,1,2
3,7,1,3
4,7,1,4
...,...,...,...
20009335,276,1,620635
20009336,276,1,620636
20009337,276,1,620637
20009338,276,1,620638


In [55]:
overview.to_parquet(file_path_prefix + "overview.parquet")